## Imports

In [1]:
import torch
from torch import nn, optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

from urllib.request import urlretrieve
from zipfile import ZipFile
import os

## Dataset Downloading

In [7]:
DATASET_ZIP_URL = 'https://www.kaggle.com/api/v1/datasets/download/masoudnickparvar/brain-tumor-mri-dataset'
LOCAL_ZIP_FILENAME = 'dataset.zip'

try:
    urlretrieve(DATASET_ZIP_URL, LOCAL_ZIP_FILENAME)
    print(f"The dataset ZIP file was successfully downloaded to '{LOCAL_ZIP_FILENAME}' file")
except Exception as e:
    print(f"Error downloading file: {e}")
    exit()

The dataset ZIP file was successfully downloaded to 'dataset.zip' file


In [8]:
EXTRACT_ZIP_DIR = 'data'

try:
    if not os.path.isdir(EXTRACT_ZIP_DIR):
        with ZipFile(LOCAL_ZIP_FILENAME, 'r') as zip_file:
            zip_file.extractall(EXTRACT_ZIP_DIR)
        print(f"The contents of the '{LOCAL_ZIP_FILENAME}' file were successfully extracted to '{EXTRACT_ZIP_DIR}' directory")
    else:
        print(f"Directory with the name '{EXTRACT_ZIP_DIR}' already exists")
except Exception as e:
    print(f"Error extracting ZIP file: {e}")

The contents of the 'dataset.zip' file were successfully extracted to 'data' directory


In [9]:
try:
    os.remove(LOCAL_ZIP_FILENAME)
    print(f"Downloaded dataset ZIP file {LOCAL_ZIP_FILENAME} was successfully removed")
except Exception as e:
    print(f"Error removing ZIP file: {e}")

Downloaded dataset ZIP file dataset.zip was successfully removed


# Device Setting

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Dataset Transformations

In [ ]:
tf = transforms.Compose([
    transforms.Resize(128, 128),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# Dataset Loaders

In [ ]:
train_loader = DataLoader(
    dataset=datasets.ImageFolder(root=EXTRACT_ZIP_DIR + '/Training', transform=tf),
    batch_size=32,
    shuffle=True,
    num_workers=4,
    pin_memory=True
)

test_loader = DataLoader(
    dataset=datasets.ImageFolder(root=EXTRACT_ZIP_DIR + '/Testing', transform=tf),
    batch_size=32,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

# Model Creation (CNN)

In [ ]:
model = nn.Sequential(
    nn.Conv2d(3, 32, 3, 1, 1), nn.ReLU(), nn.MaxPool2d(2),
    nn.Conv2d(32, 64, 3, 1, 1), nn.ReLU(), nn.MaxPool2d(2),
    nn.Conv2d(64, 128, 3, 1, 1), nn.ReLU(), nn.MaxPool2d(2),
    nn.Flatten(),
    nn.Linear(128 * 16 * 16, 256), nn.ReLU(), nn.Dropout(0.5),
    nn.Linear(256, 4) # 4 classes
).to(device)

# Loss and Optimizer

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Model Training

In [ ]:
model.train()

for epoch in range(30):
    running_loss = 0

    for x, y in train_loader:
        x, y = x.to(device), y.to(device)

        # Forward pass
        output = model(x)
        loss = loss_fn(output, y)

        # Backward and optimize
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(f'Epoch {epoch + 1}, Loss: {running_loss}')


# Model Evaluation

In [ ]:
model.eval()

with torch.no_grad():
    n_correct = 0
    n_samples = len(test_loader.dataset)

    for data, targets in test_loader:
        data, targets = data.to(device), targets.to(device)

        output = model(data)